# Quantum Support Vector Machine

This notebook is similar to `qsvm_haar_obs`, but we use pennylane instead of qiskit for the quantum circuit.\
The same general observation are valid here so we suggest to first read that notebook and than come back here to have a implementation of a qsvm in pennylane.


In [20]:
# imports

import numpy as np

from sklearn.svm import SVC
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import pennylane as qml
from pennylane.templates import AngleEmbedding, IQPEmbedding

import matplotlib.pyplot as plt
import pandas as pd

np.random.seed(42)

# Dataset

In [23]:
# Number of total exemples 
n_exemples = 300

# Loading dataset with pandas
dataset = pd.read_csv('../datasets/ds_haar_obs.csv')
dataset = dataset[:n_exemples]
dataset

,P11,P22,P33,P44,P13,P14,P24,P12,P23,P34,sep
0,0.363562,0.363562,0.363562,0.363562,0.363562,0.363562,0.363562,0.363562,0.363562,0.363562,True
1,0.478920,0.478920,0.478920,0.478920,0.478920,0.478920,0.478920,0.478920,0.478920,0.478920,True
2,0.176573,0.067155,0.055469,0.062003,0.256511,0.179351,0.105687,0.311796,0.154631,0.023350,False
3,0.022839,0.022839,0.022839,0.022839,0.022839,0.022839,0.022839,0.022839,0.022839,0.022839,True
4,0.139089,0.347255,0.249997,0.201476,0.491367,0.382842,0.488343,0.233877,0.272572,0.224575,False
...,...,...,...,...,...,...,...,...,...,...,...
295,0.424427,0.390971,0.369876,0.342670,0.374604,0.352137,0.370930,0.373050,0.392173,0.348748,False
296,0.005306,0.014902,0.001897,0.013286,0.482572,0.257956,0.106949,0.295851,0.251066,0.157335,False
297,0.374043,0.374043,0.374043,0.374043,0.374043,0.374043,0.374043,0.374043,0.374043,0.374043,True
298,0.178382,0.178382,0.178382,0.178382,0.178382,0.178382,0.178382,0.178382,0.178382,0.178382,True


In [24]:
# Number of features
n_features = 6

# Dividing dataset in features and label
X = dataset.to_numpy()[:,0:n_features]
y = dataset.to_numpy()[:,-1]

# Scaling the inputs 
scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)

# Redifining labels on -1,1
y_scaled = np.array(2 * (y - 0.5),dtype=int)

In [25]:
# Spliting dataset in training and test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled,train_size=0.8)
print(f"Train size: {len(X_train)}")
print(f"Test size: {len(X_test)}")

Train size: 240
Test size: 60


# QSVM model

In [27]:
# Setting the number of qbits the same as the number of features
n_qubits = len(X_train[0])

dev_kernel = qml.device("lightning.qubit", wires=n_qubits)

projector = np.zeros((2**n_qubits, 2**n_qubits))
projector[0, 0] = 1

# Definition of the quantum kernel
@qml.qnode(dev_kernel)
def kernel(x1, x2):
    """The quantum kernel."""
    AngleEmbedding(x1, wires=range(n_qubits))
    qml.adjoint(AngleEmbedding)(x2, wires=range(n_qubits))
    return qml.expval(qml.Hermitian(projector, wires=range(n_qubits)))


In [28]:
def kernel_matrix(A, B):
    """Compute the matrix whose entries are the kernel
       evaluated on pairwise data from sets A and B."""
    return np.array([[kernel(a, b) for b in B] for a in A])

# Training and Testing

In [29]:
# Training of the model
svm = SVC(kernel=kernel_matrix).fit(X_train, y_train)
# Accessing of the model
predictions = svm.predict(X_test)
accuracy_score(predictions, y_test)

0.9333333333333333

## Optional Encoding

In [21]:
@qml.qnode(dev_kernel)
def kernel(x1, x2):
    """The quantum kernel."""
    IQPEmbedding(x1, wires=range(n_qubits))
    qml.adjoint(IQPEmbedding)(x2, wires=range(n_qubits))
    return qml.expval(qml.Hermitian(projector, wires=range(n_qubits)))


In [26]:
# Training of the model
svm = SVC(kernel=kernel_matrix).fit(X_train, y_train)
# Accessing of the model
predictions = svm.predict(X_test)
accuracy_score(predictions, y_test)

0.9166666666666666